# pulsefit.ipynb
Take raw MIDAS data, then find and fit pulses

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob

import cdms

def psd(t,y):
	time_step = t[1]-t[0]
	psd = np.abs(np.fft.fft(y))**2
	freq = np.fft.fftfreq(y.size, time_step)
	idx = np.argsort(freq)
	# normalize
	# gitlab.com/supercdms/Reconstruction/BatCommon/-/blob/develop/pulse/PulseTools.cxx
	norm = 2.0 * time_step / len(t) # PulseTools puts n^-1/2 in FT term
	psd = np.sqrt(norm*psd)
	# DC and nyquist lose factor of 2
	psd[freq==0] /= np.sqrt(2)
	if len(t) % 2 == 0:
		psd[idx][-1] /= np.sqrt(2)
	return freq[freq>=0],psd[freq>=0]

# MIDAS colors + names
chs = ['PAS1','PBS1','PCS1','PDS1','PES1','PFS1','PAS2','PBS2','PCS2','PDS2','PES2','PFS2']
cs = ['#ffcc00','#00ff00','#ff00ff','#0000ff','#00ffff','#ff0000','#800080','#008000','#ff9900','#3366ff','#993300','#808000']

In [ ]:
# load data (takes a while)
datadir = '/fs/ddn/sdf/group/supercdms/data/CDMS/SLAC/OLAF6/' # S3DF data location
fns = glob.glob(datadir+'RUN00099*.mid.gz')

myreader = cdms.rawio.IO.RawDataReader(filepath=fns)
events = myreader.read_events(output_format=1, # pandas
                              skip_empty=True,
                              channel_names=['PAS2','PCS2'],
                              phonon_adctoamps=True,charge_adctovolts=True)

In [ ]:
# exploratory trace plotting, to inform pulse-finding
series = events.index[0][0]
evs = events.loc[series].index
print(len(evs))
fig,axes=plt.subplots(2,1,sharex=True)
for i in range(100): #len(evs)):
    ev = evs[i]
    for j in range(2):
        ind = [6,8][j] # do it this way for color purposes
        ch = chs[ind]
        trace = events.loc[series].loc[ev].loc[('Z1',ch)]
        tp = np.arange(len(trace)) * 1.0/6.25e5 #phonon
        axes[j].plot(tp,trace,color=cs[ind],alpha=0.3,lw=0.5) #-np.mean(trace)
plt.xlabel('time (s)')
plt.ylabel('Current (A)')

In [ ]:
# noise PSDs
psds = [[],[]]
for i in range(50): #len(evs)):
    ev = evs[i]
    for j in range(2):
        ind = [6,8][j] # do it this way for color purposes
        ch = chs[ind]
        trace = events.loc[series].loc[ev].loc[('Z1',ch)]
        tp = np.arange(len(trace)) * 1.0/6.25e5 #phonon
        f,ps = psd(tp,trace)
        psds[j].append(ps)

for j in range(2):
    ind = [6,8][j]
    ps = np.mean(psds[j],axis=0)
    plt.loglog(f,ps,color=cs[ind],lw=0.8,label=chs[ind])

plt.xlim(20,0.5*6.25e5)
plt.ylim(1e-12,1e-8)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Noise ($A/\sqrt{Hz}$)')
plt.legend()

#plt.minorticks_on()
plt.grid()
plt.grid(which='minor',alpha=.3)
plt.tight_layout()

In [ ]:
# amplitude histograms
# also store pulse-like events in numpy array for future
series = events.index[0][0]
evs = events.loc[series].index
amps = np.zeros((2,len(evs)))
traces = []
for i in range(len(evs)):
    ev = evs[i]
    pulse = False # is there a pulse in either channel?
    for j in range(2):
        ind = [6,8][j] # do it this way for color purposes
        ch = chs[ind]
        trace = events.loc[series].loc[ev].loc[('Z1',ch)]
        amp = np.min(trace)
        amps[j,i] = amp
        if amp < -1.5e-6:
            pulse = True
    # store pulses
    if pulse:
        traces.append([events.loc[series].loc[ev].loc[('Z1',chs[6])],events.loc[series].loc[ev].loc[('Z1',chs[8])]])
# and save
np.save('r99pulses.npy',traces)

# now plot
bins = np.arange(-4e-6,1e-7,1e-8)
plt.hist(amps[0],bins=bins,histtype='step',color=cs[6])
plt.hist(amps[1],bins=bins,histtype='step',color=cs[8])
plt.yscale('log')
plt.ylim(.5,1e4)
plt.title('Run 99 ({0} events)'.format(len(evs)))
plt.ylabel('Events')
plt.xlabel('Amplitude (A)')

In [ ]:
# reload traces, define functions, import minimize
from scipy.optimize import minimize

try:
    traces
except:
    traces = np.load('r99pulses.npy')

# 2-pole pulse function
def pulse(t,p):
    a,b,t0,lt1,lt2 = p
    t1 = 10**lt1 # convert log10(t1)
    t2 = 10**lt2 # convert log10(t2)
    norm = (t2/t1)**(t2/(t1-t2)) - (t2/t1)**(t1/(t1-t2)) # Mathematica
    y = a/norm * (np.exp(-(t-t0)/t1)-np.exp(-(t-t0)/t2)) + b
    #y = (y * np.heaviside(t-t0,1)) + b
    y[t<t0] = b
    return y

# constant-fraction discriminator
def cfd(y,frac=0.2,verbose=True):
    ipeak = np.argmin(y)
    thresh = 0.2*y[ipeak]
    for i in range(ipeak):
        if y[ipeak-i] > thresh:
            return ipeak-i
    if verbose:
        print('CFD failed')
    return -1

# 3-pole
def pulse3(t,p):
    a,b,c,t0,lt1,lt2,lt3 = p
    t1 = 10**lt1
    t2 = 10**lt2
    t3 = 10**lt3
    y = (-a+b)*np.exp(-(t-t0)/t1) + c
    y+= a*np.exp(-(t-t0)/t2)
    y-= b*np.exp(-(t-t0)/t3)
    y[t<t0] = c
    return y

# 4-pole
def pulse4(t,p):
    a,b,c,d,t0,lt1,lt2,lt3,lt4 = p
    t1 = 10**lt1
    t2 = 10**lt2
    t3 = 10**lt3
    t4 = 10**lt4
    y = (-a+b+c)*np.exp(-(t-t0)/t1) + d
    y+= a*np.exp(-(t-t0)/t2)
    y-= b*np.exp(-(t-t0)/t3)
    y-= c*np.exp(-(t-t0)/t4)
    y[t<t0] = d
    return y

In [ ]:
# fit a PAS2 pulse (2-pole)
y = traces[193][0]*1e6 # ev2, PAS2 #71 193
t = np.arange(len(y)) * 1.0/6.25e5
plt.figure(figsize=(8,4))
plt.plot(t,y,color=cs[6],label=chs[6])

# fit
pulse_ind = cfd(y)
t0 = t[pulse_ind]
y = y[(t>t0-0.001)*(t<t0+0.005)]
t = t[(t>t0-0.001)*(t<t0+0.005)]
x0 = (np.min(y),0,t0,-4,-3.9)
minfunc = lambda x: np.sum((y-pulse(t,x))**2)
res = minimize(minfunc,x0=x0,method='Nelder-Mead')

#plt.plot(t,pulse(t,x0),'k:',label='guess')
plt.plot(t,pulse(t,res.x),'k-',label='2-pole fit',lw=0.8)

plt.text(t0-5e-4,-0.5,'A = {0:.3f} $\mu$A'.format(res.x[0]))
plt.text(t0-5e-4,-0.7,r'$\tau_1$ = {0:.3f} $\mu$s'.format(1e6* 10**res.x[3]))
plt.text(t0-5e-4,-0.9,r'$\tau_2$ = {0:.3f} ms'.format(1e3* 10**res.x[4]))

#plt.xlim(.062,.064)
plt.xlim(.0864,.088)
plt.xlabel('Time (s)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend()
plt.tight_layout()

In [ ]:
# plot residuals
ax=plt.subplot(211)
plt.plot(t*1e3,y,color=cs[6],label=chs[6])
plt.plot(t*1e3,pulse(t,res.x),'k-',lw=0.8,label='Fit (2-pole)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend(loc=4)
plt.subplot(212,sharex=ax)
plt.plot(t*1e3,pulse(t,res.x)-y,color=cs[6],label='Residuals')
plt.axhline(0,color='k',alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel(r'Fit - Data ($\mu$A)')
#plt.xlim(62.25,63.25)
plt.xlim(86.4,88.0)
plt.legend()

In [ ]:
# compare 3-pole residuals
y = traces[71][0]*1e6 # ev2, PAS2 #71 193
t = np.arange(len(y)) * 1.0/6.25e5

# fit
pulse_ind = cfd(y)
t0 = t[pulse_ind]
y = y[(t>t0-0.001)*(t<t0+0.005)]
t = t[(t>t0-0.001)*(t<t0+0.005)]
x0 = (np.min(y),1.0,0,t0,-4,-3.9,-3.8)
minfunc = lambda x: np.sum((y-pulse3(t,x))**2)
res = minimize(minfunc,x0=x0,method='Nelder-Mead')
print(res.x)

ax=plt.subplot(211)
plt.plot(t*1e3,y,color=cs[6],label=chs[6])
#plt.plot(t*1e3,pulse3(t,x0),'k:',lw=0.8)
plt.plot(t*1e3,pulse3(t,res.x),'k-',lw=0.8,label='Fit (3-pole)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend(loc=4)
plt.subplot(212,sharex=ax)
plt.plot(t*1e3,pulse3(t,res.x)-y,color=cs[6],label='Residuals')
plt.axhline(0,color='k',alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel(r'Fit - Data ($\mu$A)')
plt.xlim(62.25,63.25)
plt.legend()


In [ ]:
# fit PCS2 pulse

y = traces[19][1]*1e6 # ev19, PCS2
t = np.arange(len(y)) * 1.0/6.25e5
plt.figure(figsize=(8,4))
plt.plot(t,y,color=cs[8],label=chs[8])

# fit
pulse_ind = cfd(y)
t0 = t[pulse_ind]
y = y[(t>t0-0.001)*(t<t0+0.005)]
t = t[(t>t0-0.001)*(t<t0+0.005)]
x0 = (np.min(y),0,t0,-4,-3.9)
minfunc = lambda x: np.sum((y-pulse(t,x))**2)
res = minimize(minfunc,x0=x0,method='Nelder-Mead')

#plt.plot(t,pulse(t,x0),'k:',label='guess')
plt.plot(t,pulse(t,res.x),'k-',label='2-pole fit',lw=0.8)

plt.text(t0-5e-4,-0.5,'A = {0:.3f} $\mu$A'.format(res.x[0]))
plt.text(t0-5e-4,-0.7,r'$\tau_1$ = {0:.3f} $\mu$s'.format(1e6* 10**res.x[3]))
plt.text(t0-5e-4,-0.9,r'$\tau_2$ = {0:.3f} ms'.format(1e3* 10**res.x[4]))

plt.xlim(.013,.015)
plt.xlabel('Time (s)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend()
#plt.tight_layout()
print(res.x)

In [ ]:
# 2-pole residuals
ax=plt.subplot(211)
plt.plot(t*1e3,y,color=cs[8],label=chs[8])
#plt.plot(t*1e3,pulse(t,x0),'k:',lw=0.8)
plt.plot(t*1e3,pulse(t,res.x),'k-',lw=0.8,label='Fit (2-pole)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend(loc=4)
plt.subplot(212,sharex=ax)
plt.plot(t*1e3,pulse(t,res.x)-y,color=cs[8],label='Residuals')
plt.axhline(0,color='k',alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel(r'Fit - Data ($\mu$A)')
plt.xlim(13.55,14)
plt.legend()

In [ ]:
# PCS2 compare 2-, 3-, 4-pole

y = traces[19][1]*1e6 # ev19, PCS2
t = np.arange(len(y)) * 1.0/6.25e5
pulse_ind = cfd(y)
t0 = t[pulse_ind]
mask = (t>t0-0.0001)*(t<t0+0.0005)
y = y[mask]
t = t[mask]

# fit
x0 = (np.min(y),0.03,t0,-5,-4)
minfunc = lambda x: np.sum((y-pulse(t,x))**2)
res2 = minimize(minfunc,x0=x0,method='Nelder-Mead')
print(res2.x)
print('2-pole:',res2.fun)

x3 = (np.min(y),0.5,0.02,t0,-5,-4,-3.5)
minfunc = lambda x: np.sum((y-pulse3(t,x))**2)
res3 = minimize(minfunc,x0=x3,method='Nelder-Mead')
print(res3.x)
print('3-pole:',res3.fun)

x4 = (np.min(y),0.5,0.3,0,t0,-5,-4,-3.5,-3.2)
minfunc = lambda x: np.sum((y-pulse4(t,x))**2)
res4 = minimize(minfunc,x0=x4,method='Nelder-Mead')
print(res4.x)
print('4-pole:',res4.fun)

ax=plt.subplot(211)
plt.plot(t*1e3,y,color='k',label=chs[8])
plt.plot(t*1e3,pulse(t,res2.x),color=cs[9],lw=0.8,label='Fit (2-pole)')
plt.plot(t*1e3,pulse3(t,x3),'k:') # show initial guess
plt.plot(t*1e3,pulse3(t,res3.x),color=cs[8],lw=0.8,label='Fit (3-pole)')
#plt.plot(t*1e3,pulse4(t,x4),'k:') # show initial guess
plt.plot(t*1e3,pulse4(t,res4.x),color=cs[7],lw=0.8,label='Fit (4-pole)')
plt.ylabel(r'Current ($\mu$A)')
plt.legend(loc=4)
plt.subplot(212,sharex=ax)
plt.plot(t*1e3,pulse(t,res2.x)-y,color=cs[9],label='2 Residuals')
plt.plot(t*1e3,pulse3(t,res3.x)-y,color=cs[8],label='3 Residuals')
plt.plot(t*1e3,pulse4(t,res4.x)-y,color=cs[7],label='4 Residuals')
plt.axhline(0,color='k',alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel(r'Fit - Data ($\mu$A)')
plt.xlim(13.55,14.1)
plt.ylim(-.2,.2)
plt.legend()

The 2-pole fit is not improved on significantly.

In [ ]:
# print a bunch of fit parameters
for i in range(len(traces)):
    y = traces[i][0]*1e6
    t = np.arange(len(y)) * 1.0/6.25e5
    pulse_ind = cfd(y,verbose=False)
    if pulse_ind == -1:
        continue
    t0 = t[pulse_ind]
    mask = (t>t0-0.0001)*(t<t0+0.0005)
    y = y[mask]
    t = t[mask]

    peak = np.min(y)
    if peak < -2.4 or peak > -1.5:
        continue

    # fit
    x0 = (peak,0.01,t0,-5,-3.9)
    minfunc = lambda x: np.sum((y-pulse(t,x))**2)
    res2 = minimize(minfunc,x0=x0,method='Nelder-Mead')
    vals = [res2.x[0]-res2.x[1], 1e6*10**res2.x[3], 1e3*10**res2.x[4]]
    print('{0:.3f}   {1:.3f} us   {2:.3f} ms  '.format(*vals),res2.fun)   

Summary:
* PAS2 tau1 ~ 6 us, tau2 ~ 0.2 ms
* PCS2 tau1 ~ 7 us, tau2 ~ 0.1 ms